# Advent of Code Day 14
#### Problem in full can be found here: https://adventofcode.com/2023/day/14

In [139]:
# In an input file with 'O', '#', '.' where 'O' is a boulder, '#' is a cube-shaped rock, '.' is a empty space, figure out the sum
# of all the boulder's new locations after the map gets tilted which causes the boulders to roll North and the rocks to stay still
# (a boulder can't roll over a rock)
import numpy as np

# Gather the input from file and store in 2D array
file = open("inputday14.txt")
map = []
for line in file:
    line = line.strip()
    map.append(list(line))
file.close()

# Take the transpose of the map then push the boulders North
map = transposeMap(map, "N")

# Part 1
# Calculate the load for each row
sum = 0
for row in map:
    sum += calculateLoad(row)
print(sum)

# Part 2 involves cycling and 1 cycle shifts everything North -> West -> South -> East so find the sum of the new locations after
# 1000000000 cycles

# Finish the current cycle
map = transposeMap(map, "W")
map = transposeMap(map, "S")
map = transposeMap(map, "E")

# Figure out the cycle pattern
cycles = []
# Loop until the same map is found in the cycles
while map not in cycles:
    # Add the current finished map cycle to the list of cycles
    cycles.append(map)
    # Run another set of cycles
    map = transposeMap(map, "N")
    map = transposeMap(map, "W")
    map = transposeMap(map, "S")
    map = transposeMap(map, "E")

# Calculate the loop length by taking the length of the amount of cycles gone through - the first occurrence of the 
# current map as this is now our second occurrence
loop_length = len(cycles) - cycles.index(map)
# Take out all the elements fonud in the cycle that aren't part of the loop 
not_in_loop = cycles.index(map)
# Find what iteration of the cycle we'll be on at the 1000000000th mark
cycles_left = (1000000000 - not_in_loop) % loop_length

# Complete the cycles necessary
for i in range(cycles_left - 1):
    map = transposeMap(map, "N")
    map = transposeMap(map, "W")
    map = transposeMap(map, "S")
    map = transposeMap(map, "E")

# Convert the map back to it's transpose since we want it in a row format
convertedMap = np.array(map)
map = convertedMap.T.tolist()
# Calculate the load for each row
sum = 0
for row in map:
    sum += calculateLoad(row)
print(sum)

110779
86069


In [85]:
# Function that calculates the load of a given row
def calculateLoad(row):
    sum = 0
    length = len(row)
    
    for i in range(length):
        # If a boulder
        if row[i] == 'O':
            sum += (length - i)

    # Return the sum
    return sum

In [132]:
# A function that shifts the boulders in a given row in the given direction
def shiftRow(row, direction):
    length = len(row)
    if direction == "N" or direction == "W":
        rowOrder = range(length)
    else:
        rowOrder = range(length-1, -1, -1)

    for i in rowOrder:
        # If a boulder
        if row[i] == 'O':
            newIndex = i
            
            # Roll the boulder as far as it can go
            while newIndex > 0 and row[newIndex-1] == '.' and rowOrder == range(length):
                newIndex -= 1
            while newIndex < length-1 and row[newIndex+1] == '.' and rowOrder == range(length-1, -1, -1):
                newIndex += 1

            # Update the boulders location
            row[i] = '.'
            row[newIndex] = 'O'

    # Return the row
    return row

In [134]:
# A function that transposes the map then updates the boulders to be pushed in the given direction
def transposeMap(map, direction):
    # Convert the map into a numpy array
    convertedMap = np.array(map)
    # Take the transpose then convert back to a list
    map = convertedMap.T.tolist()
    # Update each row in the map to shift in the given direction
    for i in range(len(map)):
        map[i] = shiftRow(map[i], direction)
    # Return the map
    return map